# 1. Introducción

El primer paso es hacer cambios en el dataset original del INDEC para realizar un EDA posteriormente.

El dataset original de la página del INDEC es un archivo .xls de una versión vieja del programa.

Hice pequeños cambios de diseño con Excel. Como último paso genero un archivo unico .csv por cada hoja para mi conveniencia.

***Fuente de los datasets del proyecto:***

https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-5-31

https://www.indec.gob.ar/indec/web/Institucional-Indec-Codgeo

https://www.datos.gob.ar/series/api/series/?ids=168.1_T_CAMBIOR_D_0_0_26

In [ ]:
# Importo las librerias necesarias para esta parte.
import pandas as pd # Para usar DataFrames.

In [ ]:
# Leo todas las hojas del archivo .xlsx y las convierto en DataFrames. (Todas las hojas del archivo .xlsx tienen el mismo formato de columnas y filas)
df_var_mensual = pd.read_excel('../Data/Dataset_IPC_INDEC.xlsx', sheet_name = 'Variacion_mensual_IPC')
df_var_interanual = pd.read_excel('../Data/Dataset_IPC_INDEC.xlsx', sheet_name = 'Variacion_interanual_IPC')
df_indices = pd.read_excel('../Data/Dataset_IPC_INDEC.xlsx', sheet_name = 'Indices_IPC')

In [ ]:
# Veo las primeras filas del DataFrame.
df_var_mensual.head()

In [ ]:
# Veo la info del DataFrame.
df_var_mensual.info()

In [ ]:
# En esta celda voy a trasponer las columnas de los meses para tener solo una columna nueva llamada 'Fecha' con todas las fechas en filas,
# clasificadas por 'Region' y 'Division_IPC'.

# Lista de los 3 DataFrames originales que despues voy a guardar en archivos .csv.
dataframes = [df_var_mensual, df_var_interanual, df_indices]

# Lista para almacenar los resultados de la trasposicion.
df_traspuestos = []

# Itero sobre los 3 DataFrames y aplico la funcion melt a cada uno.
for df in dataframes:
    columnas_meses = df.columns[2:]  # Lista de las columnas de meses.
    df_traspuesto = df.melt(id_vars=['Region', 'Division_IPC'], value_vars=columnas_meses, var_name='Fecha', value_name='Valor_IPC')
    df_traspuestos.append(df_traspuesto)

In [ ]:
# Asigno los nuevos DataFrames.
df_var_mensual_csv = df_traspuestos[0]
df_var_interanual_csv = df_traspuestos[1]
df_indices_csv = df_traspuestos[2]

In [ ]:
# Veo si quedaron las columnas y las filas del modo que necesito.
df_var_mensual_csv.head()

In [ ]:
# Veo la informacion general de los nuevos DataFrames. (Para verificar si los tipos de datos de las columnas estan bien)
print('Info del "df_var_mensual_csv":\n')
df_var_mensual_csv.info()

print('\nInfo del "df_var_interanual_csv":\n')
df_var_interanual_csv.info()

print('\nInfo del "df_indices_csv":\n')
df_indices_csv.info()

In [ ]:
# Guardo los 3 DataFrames nuevos en archivos .csv.
df_var_mensual_csv.to_csv('../Data/Variacion_mensual_IPC.csv', index = False)
df_var_interanual_csv.to_csv('../Data/Variacion_interanual_IPC.csv', index = False)
df_indices_csv.to_csv('../Data/Indices_IPC.csv', index = False)

**Nota:** Todas estas celdas de código ya fueron ejecutadas y los archivos .csv nuevos estan guardados en la carpeta 'Data' del proyecto.

# 2. EDA - Exploratory Data Analysis

In [ ]:
# Importo las librerias necesarias para el proyecto.
import pandas as pd # Para usar DataFrames.
import numpy as np # Para analisis de numeros.
from scipy import stats # Para analisis estadistico.

# Para graficar.
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sb
import itertools
from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline

# Para modelos automaticos.
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.model_selection import train_test_split

# Para analisis de datos geoespaciales.
import geopandas as gpd

# Otras librerias.
import warnings
warnings.filterwarnings("ignore") # Ignoro las advertencias.

In [ ]:
# Leo todas las hojas del archivo excel y las convierto en DataFrames.
df_var_mensual = pd.read_csv('../Data/Variacion_mensual_IPC.csv', parse_dates=['Fecha'])
df_var_interanual = pd.read_csv('../Data/Variacion_interanual_IPC.csv', parse_dates=['Fecha'])
df_indices = pd.read_csv('../Data/Indices_IPC.csv', parse_dates=['Fecha'])

In [ ]:
# Veo las primeras filas del DataFrame.
df_var_mensual.head()

In [ ]:
# Veo la info del DataFrame.
df_var_mensual.info()

In [ ]:
# Obtengo los valores unicos de la columna 'Region'.
regiones = df_var_mensual['Region'].unique()

# Imprimo los valores unicos por pantalla.
print('Regiones estadísticas del INDEC:\n')
for valor_reg in regiones:
    print('-', valor_reg)

# Obtengo los valores unicos de la columna 'Division_IPC'.
divisiones = df_var_mensual['Division_IPC'].unique()

# Imprimo los valores unicos por pantalla.
print('\nDivisiones del IPC:\n')
for valor_div in divisiones:
    print('-', valor_div)

In [ ]:
# Filtro los datos para los ultimos 12 meses en todo el pais de la division general del IPC.
df_graf1 = df_var_mensual[(df_var_mensual['Fecha'] >= '2022-06-01') & (df_var_mensual['Region'] == 'Nacional') & (df_var_mensual['Division_IPC'] != 'Nivel general')]

# Encuentro la division del IPC mas alta para cada mes.
df_max_division = df_graf1.groupby(df_graf1['Fecha'].dt.to_period('M')).apply(lambda x: x.loc[x['Valor_IPC'].idxmax()])

# Obtengo la lista de divisiones unicas del DataFrame 'df_max_division'.
divisiones_unicas = df_max_division['Division_IPC'].unique()

# Creo un diccionario para asignar colores a cada division del IPC.
colores = dict(zip(divisiones_unicas, ['blue', 'red', 'yellow', 'green', 'orange', 'purple', 'grey', 'black']))

# Creo la figura del grafico.
fig = go.Figure()

# Itero sobre cada division.
for division in divisiones_unicas:
    # Filtro el DataFrame para obtener solo las filas correspondientes a la division actual.
    df_division = df_max_division[df_max_division['Division_IPC'] == division]
    
    # Obtengo el color correspondiente a la division.
    color = colores[division]
    
    # Agrego una barra para cada division.
    fig.add_trace(go.Bar(x=df_division['Fecha'],
                         y=df_division['Valor_IPC'],
                         marker_color=color,
                         text=df_division['Valor_IPC'],
                         textposition='auto',
                         hovertemplate='%{y:.1f}',
                         hoverlabel=dict(bgcolor='white'),
                         name=division))

# Configuro el disenio del grafico.
fig.update_layout(title='División del IPC con mayor variación mensual - Total Nacional',
                  xaxis=dict(title='', tickformat='%b %y', dtick='M1'),
                  yaxis=dict(title='Variación mensual'),
                  hovermode='x unified',
                  showlegend=True,
                  height=600,
                  width=1090)

# Muestro el grafico interactivo.
fig.show()

In [ ]:
# Filtro los datos para ver los de todo el pais de la division general del IPC.
df_graf2_a = df_var_mensual[(df_var_mensual['Region'] == 'Nacional') & (df_var_mensual['Division_IPC'] == 'Nivel general')]

# Filtro los datos 12 meses (1 año) para atras desde el ultimo dato mensual.
df_graf2_a = df_graf2_a[(df_graf2_a['Fecha'] >= '2022-06-01') & (df_graf2_a['Fecha'] <= '2023-05-31')]

# Filtro los datos para ver los de todo el pais de la division general del IPC.
df_graf2_b = df_var_interanual[(df_var_interanual['Region'] == 'Nacional') & (df_var_interanual['Division_IPC'] == 'Nivel general')]

# Filtro los datos 12 meses (1 año) para atras desde el ultimo dato mensual.
df_graf2_b = df_graf2_b[(df_graf2_b['Fecha'] >= '2022-06-01') & (df_graf2_b['Fecha'] <= '2023-05-31')]

# Creo la figura del grafico.
fig = go.Figure()

# Agrego la curva de variacion mensual al grafico.
fig.add_trace(go.Scatter(x=df_graf2_a['Fecha'],
                         y=df_graf2_a['Valor_IPC'],
                         mode='lines+markers',
                         line=dict(color='steelblue'),
                         text=df_graf2_a['Valor_IPC'],
                         textposition='top center',
                         hovertemplate='%{y:.1f}',
                         hoverlabel=dict(bgcolor='white'),
                         name='Variación mensual'))

# Agrego la curva de variacion interanual al grafico.
fig.add_trace(go.Scatter(x=df_graf2_b['Fecha'],
                         y=df_graf2_b['Valor_IPC'],
                         mode='lines+markers',
                         line=dict(color='red'),
                         text=df_graf2_b['Valor_IPC'],
                         textposition='top center',
                         hovertemplate='%{y:.1f}',
                         hoverlabel=dict(bgcolor='white'),
                         name='Variación interanual',
                         yaxis='y2'))

# Configuro el disenio del grafico.
fig.update_layout(title='Variación mensual e interanual del Nivel general del IPC - Total Nacional',
                  xaxis=dict(title='', tickformat='%b %y', dtick='M1'),
                  yaxis=dict(title='Variación mensual'),
                  yaxis2=dict(title='Variación interanual', overlaying='y', side='right'),
                  hovermode='x unified',
                  showlegend=True,
                  height=600,
                  width=1090)

# Muestro el grafico interactivo.
fig.show()

In [ ]:
# Filtro los datos para el total del pais y la division general del IPC.
df_graf3_a = df_var_mensual[(df_var_mensual['Region'] == 'Nacional') & (df_var_mensual['Division_IPC'] == 'Nivel general')]

# Filtro los datos para el total del pais y la division general del IPC.
df_graf3_b = df_indices[(df_indices['Region'] == 'Nacional') & (df_indices['Division_IPC'] == 'Nivel general')]

# Creo la figura del grafico.
fig = go.Figure()

# Agrego la curva de variacion mensual al grafico.
fig.add_trace(go.Scatter(x=df_graf3_a['Fecha'],
                         y=df_graf3_a['Valor_IPC'],
                         line=dict(color='steelblue'),
                         text=df_graf3_a['Valor_IPC'],
                         textposition='top center',
                         hovertemplate='%{y:.1f}',
                         hoverlabel=dict(bgcolor='white'),
                         name='Variación mensual'))

# Agrego la curva de indice del IPC al grafico.
fig.add_trace(go.Scatter(x=df_graf3_b['Fecha'],
                         y=df_graf3_b['Valor_IPC'],
                         line=dict(color='red'),
                         text=df_graf3_b['Valor_IPC'],
                         textposition='top center',
                         hovertemplate='%{y:.1f}',
                         hoverlabel=dict(bgcolor='white'),
                         name='Índice acumulado',
                         yaxis='y2'))

# Configuro el disenio del grafico.
fig.update_layout(title='Variación mensual e Índice acumulado del Nivel general del IPC - Total Nacional',
                  xaxis=dict(title='', tickformat='%b %y', dtick='M12'),
                  yaxis=dict(title='Variación mensual'),
                  yaxis2=dict(title='Índice acumulado', overlaying='y', side='right'),
                  hovermode='x unified',
                  showlegend=True,
                  height=600,
                  width=1090)

# Muestro el grafico interactivo.
fig.show()

# 3. Time Series Analysis & Forecasting

In [ ]:
# Filtro los datos para el total del pais y la division general del IPC.
df_modelos = df_var_mensual[(df_var_mensual['Region'] == 'Nacional') & (df_var_mensual['Division_IPC'] == 'Nivel general')]

# Elimino las columnas 'Region' y 'Division_IPC' del DataFrame para quedarme con las necesarias ('Fecha' y 'Valor_IPC').
df_modelos = df_modelos.drop(['Region', 'Division_IPC'], axis=1)

# Pongo la columna 'Fecha' como el nuevo indice del DataFrame.
df_modelos.set_index(['Fecha'],inplace=True)

df_modelos.head()

In [ ]:
# Verifico si hay nulos en el DataFrame.
print(df_modelos.isna().sum())

In [ ]:
# Divido el DataFrame original en un train y un test.
df_train, df_test = train_test_split(df_modelos, test_size=0.15, shuffle=False)

# Vemos los últimos datos del set de entrenamiento:
print('Últimos datos del set de entrenamiento:\n')
print(df_train.tail())

# Vemos los primeros datos del set de testeo:
print('\nPrimeros datos del set de testeo:\n')
print(df_test.head())

In [ ]:
# Creo una figura con subplots.
fig, axes = plt.subplots(1, 2,figsize=(14,5))

# Grafico de la serie original en el primer subplot.
axes[0].plot(df_train.Valor_IPC)
axes[0].set_title('Serie original')

# Grafico de la ACF en el segundo subplot.
plot_acf(df_train.Valor_IPC, ax=axes[1])

plt.show()

Desciende a cero rápidamente, por lo tanto la serie es estacionaria.

In [ ]:
# Realizo el ADF Test en la serie de datos 'df.Valor_IPC' para verificar si realmente es estacionaria.
result = adfuller(df_train['Valor_IPC'])

# Imprimo la estadistica ADF.
print('ADF Statistic:', result[0].round(4))

# Imprimo el p-value.
print('p-value:', result[1].round(4))

Se observa un p-value <= 0.05 para la serie. Por lo tanto, es estacionaria.

No hay que realizar ninguna diferenciación, entonces $d = 0$

In [ ]:
# Creo una figura con subplots.
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Grafico de la ACF en el primer subplot.
plot_acf(df_train, ax=axes[0], title='ACF')

# Grafico de la PACF en el segundo subplot.
plot_pacf(df_train, ax=axes[1], title='PACF')

# Muestro el grafico.
plt.show()

Este gráfico para buscar los valores de p y q es muy subjetivo.

A continuación voy a realizar una iteración en un rango de valores para encontrar los mejores valores con mas precisión.

### Modelo ARIMA

In [ ]:
# Defino el rango de valores para p, d y q.
p_values = range(0,7)
d_values = [0]
q_values = range(0,7)

# Genero todas las combinaciones posibles de valores.
parameter_combinations = list(itertools.product(p_values, d_values, q_values))

# Inicializo variables para almacenar los resultados.
best_aic = float('inf')
best_params = None

# Itero sobre todas las combinaciones de valores.
for params in parameter_combinations:
    p, d, q = params
    model = ARIMA(df_train['Valor_IPC'], order=(p, d, q))
    result = model.fit()
    aic = result.aic
    
    # Verifico si el AIC actual es el mejor hasta ahora.
    if aic < best_aic:
        best_aic = aic
        best_params = params
        
    # Imprimo el progreso en porcentaje.
    print(round((parameter_combinations.index(params) + 1) / len(parameter_combinations) * 100, 1), '%', end='\r')

# Imprimo el mejor valor de AIC y los correspondientes valores de p, d y q.
print(f"El mejor valor de AIC: {best_aic}")
print(f"Mejores parámetros: p={best_params[0]}, d={best_params[1]}, q={best_params[2]}")

In [ ]:
# Creo el modelo ARIMA utilizando los mejores parametros encontrados.
model_ARIMA = ARIMA(df_train, order=(best_params[0], best_params[1], best_params[2]))

# Ajusto el modelo ARIMA a los datos de entrenamiento.
result_ARIMA = model_ARIMA.fit()

# Imprimo un resumen del modelo ajustado.
print(result_ARIMA.summary())

In [ ]:
# Creo un DataFrame con los residuales del modelo ARIMA.
residuals_ARIMA = pd.DataFrame(result_ARIMA.resid)

# Creo una figura para mostrar multiples graficos.
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Grafico los residuales.
residuals_ARIMA.plot(title="Residuals", ax=axes[0, 0])

# Grafico la densidad de los residuales.
residuals_ARIMA.plot(kind='kde', title='Density', ax=axes[0, 1])

# Grafico la (ACF) de los residuales.
plot_acf(residuals_ARIMA, ax=axes[1, 0], title='ACF')

# Grafico la (PACF) de los residuales.
plot_pacf(residuals_ARIMA, ax=axes[1, 1], title='PACF')

# Muestro la figura con los graficos de los residuales.
plt.show()

In [ ]:
# Realizo predicciones utilizando el modelo ARIMA ajustado.
prediction_ARIMA = result_ARIMA.forecast(len(df_test['Valor_IPC']))

prediction_ARIMA

In [ ]:
# Creo la figura del grafico.
fig = go.Figure()

# Agrego una linea de los datos originales al grafico.
fig.add_trace(go.Scatter(x=df_modelos.index, y=df_modelos['Valor_IPC'], mode='lines', name='Datos originales', line=dict(color='steelblue')))

# Agrego una linea de la prediccion al grafico.
fig.add_trace(go.Scatter(x=df_test.index, y=prediction_ARIMA, mode='lines', name='Predicción', line=dict(color='green')))

# Actualizo el disenio del grafico.
fig.update_layout(title='Variación mensual del Nivel general del IPC - Total Nacional', xaxis_title='', yaxis_title='Variación mensual')

# Muestro el grafico interactivo.
fig.show()

### Modelo SARIMA

In [ ]:
# Defino el rango de valores para p, d, q, P, D, Q y s.
p_values = range(0, 3)
d_values = [0]
q_values = range(0, 3)
P_values = range(0, 3)
D_values = range(0, 2)
Q_values = range(0, 3)
s_values = [12]

# Genero todas las combinaciones posibles de valores.
parameter_combinations = list(itertools.product(p_values, d_values, q_values, P_values, D_values, Q_values, s_values))

# Inicializo variables para almacenar los resultados.
best_aic = float('inf')
best_params = None

# Itero sobre todas las combinaciones de valores.
for params in parameter_combinations:
    p, d, q, P, D, Q, s = params
    model = SARIMAX(df_train['Valor_IPC'], order=(p, d, q), seasonal_order=(P, D, Q, s))
    result = model.fit()
    aic = result.aic
    
    # Verifico si el AIC actual es el mejor hasta ahora.
    if aic < best_aic:
        best_aic = aic
        best_params = params
        
    # Imprimo el progreso en porcentaje.
    print(round((parameter_combinations.index(params) + 1) / len(parameter_combinations) * 100, 1), '%', end='\r')

# Imprimo el mejor valor de AIC y los correspondientes valores de p, d, q, P, D, Q y s.
print(f"El mejor valor de AIC: {best_aic}")
print(f"Mejores parámetros: p={best_params[0]}, d={best_params[1]}, q={best_params[2]}, P={best_params[3]}, D={best_params[4]}, Q={best_params[5]}, s={best_params[6]}")

In [ ]:
# Creo el modelo SARIMA utilizando los mejores parametros encontrados.
model_SARIMA = SARIMAX(df_train, order=(best_params[0], best_params[1], best_params[2]), seasonal_order=(best_params[3], best_params[4], best_params[5], best_params[6]))

# Ajusto el modelo SARIMA a los datos de entrenamiento.
result_SARIMA = model_SARIMA.fit()

# Imprimo un resumen del modelo ajustado.
print(result_SARIMA.summary())

In [ ]:
# Creo un DataFrame con los residuos del modelo SARIMA.
residuals_SARIMA = pd.DataFrame(result_SARIMA.resid)

# Creo una figura para mostrar multiples graficos.
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Grafico los residuos.
residuals_SARIMA.plot(title="Residuals", ax=axes[0, 0])

# Grafico la densidad de los residuos.
residuals_SARIMA.plot(kind='kde', title='Density', ax=axes[0, 1])

# Grafico la (ACF) de los residuos.
plot_acf(residuals_SARIMA, ax=axes[1, 0], title='ACF')

# Grafico la (PACF) de los residuo.
plot_pacf(residuals_SARIMA, ax=axes[1, 1], title='PACF')

# Muestro la figura con los graficos de los residuos.
plt.show()

In [ ]:
# Realizo predicciones utilizando el modelo SARIMA ajustado.
prediction_SARIMA = result_SARIMA.forecast(len(df_test['Valor_IPC']))

prediction_SARIMA

In [ ]:
# Creo la figura del grafico.
fig = go.Figure()

# Agrego una linea de los datos originales al grafico.
fig.add_trace(go.Scatter(x=df_modelos.index, y=df_modelos['Valor_IPC'], mode='lines', name='Datos originales', line=dict(color='steelblue')))

# Agrego una linea de la prediccion al grafico.
fig.add_trace(go.Scatter(x=df_test.index, y=prediction_SARIMA, mode='lines', name='Predicción', line=dict(color='green')))

# Actualizo el disenio del grafico.
fig.update_layout(title='Variación mensual del Nivel general del IPC - Total Nacional', xaxis_title='', yaxis_title='Variación mensual')

# Muestro el grafico interactivo.
fig.show()

### Modelo SARIMAX

In [ ]:
# Leo el archivo 'Dolar_BNA' y lo convierto en un DataFrame.
df_dolar = pd.read_csv('../Data/Dolar_BNA.csv', parse_dates=['indice_tiempo'])

# Veo las primeras filas del DataFrame.
df_dolar.head()

In [ ]:
# Veo la info del DataFrame.
df_dolar.info()

In [ ]:
# Selecciono las columnas de interes.
df_dolar = df_dolar[['indice_tiempo','tipo_cambio_bna_vendedor']]

# Cambio los nombres de las columnas.
df_dolar.columns = ['Fecha','Dolar_BNA']

df_dolar.info()

In [ ]:
# Uno el DataFrame de los modelos con el de Dolar BNA, por fecha.
df_unido = pd.merge(df_modelos, df_dolar, on='Fecha')

# Pongo la columna 'Fecha' como el nuevo indice del DataFrame.
df_unido.set_index(['Fecha'],inplace=True)

df_unido.head()

In [ ]:
# Verifico si hay nulos en el DataFrame.
df_unido.isna().sum()

In [ ]:
# Divido el DataFrame original en un train y un test.
df_train, df_test = train_test_split(df_unido, test_size=0.15, shuffle=False)

# Vemos los últimos datos del set de entrenamiento:
print('Últimos datos del set de entrenamiento:\n')
print(df_train.tail())

# Vemos los primeros datos del set de testeo:
print('\nPrimeros datos del set de testeo:\n')
print(df_test.head())

In [ ]:
# Creo una figura con subplots.
fig, axes = plt.subplots(1, 2,figsize=(14,5))

# Grafico de la serie original en el primer subplot.
axes[0].plot(df_train.Dolar_BNA)
axes[0].set_title('Serie original')

# Grafico de la ACF en el segundo subplot.
plot_acf(df_train.Dolar_BNA, ax=axes[1])

plt.show()

Desciende a cero lentamente, por lo tanto la serie de la variable exógena no es estacionaria.

In [ ]:
# Realizo el ADF Test en la serie de datos 'df.Dolar_BNA' para verificar si realmente no es estacionaria.
result = adfuller(df_train['Dolar_BNA'])

# Imprimo la estadistica ADF.
print('ADF Statistic:', result[0].round(4))

# Imprimo el p-value.
print('p-value:', result[1].round(4))

Se observa un p-value > 0.05 para la serie de la variable exógena. Por lo tanto, no es estacionaria.

Hay que realizar diferenciación.

In [ ]:
# Realizo el ADF Test en la serie de datos 'df.Dolar_BNA' con la primera diferenciacion.
result = adfuller(df_train['Dolar_BNA'].diff().dropna())

# Imprimo la estadistica ADF.
print('ADF Statistic:', result[0].round(4))

# Imprimo el p-value.
print('p-value:', result[1].round(4))

Se observa un p-value <= 0.05 para la serie de la variable exógena. Por lo tanto, ya es estacionaria.

Además, lo podemos comprobar gráficamente.

In [ ]:
# Creo una figura con subplots.
fig, axes = plt.subplots(1, 2,figsize=(14,5))

# Grafico de la primera diferenciacion en el primer subplot.
axes[0].plot(df_train.Dolar_BNA.diff().dropna())
axes[0].set_title('Diferenciación de Primer orden')

# Grafico de la ACF de la primera diferenciacion en el segundo subplot.
plot_acf(df_train.Dolar_BNA.diff().dropna(), ax=axes[1])

# Muestro la figura con los subplots.
plt.show()

Desciende a cero rápidamente, por lo tanto la serie de la variable exógena ya es estacionaria.

In [ ]:
# Defino el rango de valores para p, d, q, P, D, Q y s.
p_values = range(0, 3)
d_values = [0]
q_values = range(0, 3)
P_values = range(0, 3)
D_values = range(0, 2)
Q_values = range(0, 3)
s_values = [12]

# Genero todas las combinaciones posibles de valores.
parameter_combinations = list(itertools.product(p_values, d_values, q_values, P_values, D_values, Q_values, s_values))

# Inicializo variables para almacenar los resultados.
best_aic = float('inf')
best_params = None

# Itero sobre todas las combinaciones de valores.
for params in parameter_combinations:
    p, d, q, P, D, Q, s = params
    var_exog = df_train['Dolar_BNA']  # Agrego las columnas de las variables exogenas.
    model = SARIMAX(df_train['Valor_IPC'], exog=var_exog, order=(p, d, q), seasonal_order=(P, D, Q, s))
    result = model.fit()
    aic = result.aic

    # Verifico si el AIC actual es el mejor hasta ahora.
    if aic < best_aic:
        best_aic = aic
        best_params = params

    # Imprimo el progreso en porcentaje.
    print(round((parameter_combinations.index(params) + 1) / len(parameter_combinations) * 100, 1), '%', end='\r')

# Imprimo el mejor valor de AIC y los correspondientes valores de p, d, q, P, D, Q y s.
print(f"El mejor valor de AIC: {best_aic}")
print(f"Mejores parámetros: p={best_params[0]}, d={best_params[1]}, q={best_params[2]}, P={best_params[3]}, D={best_params[4]}, Q={best_params[5]}, s={best_params[6]}")

In [ ]:
# Creo el modelo SARIMAX con el mejor orden que encontre anteriormente.
model_SARIMAX = SARIMAX(df_train['Valor_IPC'],exog=var_exog,order=(best_params[0],best_params[1],best_params[2]),seasonal_order=(best_params[3],best_params[4],best_params[5],best_params[6]))

# Ajusto el modelo a los datos.
result_SARIMAX = model_SARIMAX.fit()

# Obtengo el resumen del modelo.
print(result_SARIMAX.summary())

In [ ]:
# Creo un DataFrame con los residuos del modelo SARIMAX.
residuals_SARIMAX = pd.DataFrame(result_SARIMAX.resid)

# Creo una figura para mostrar multiples graficos.
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Grafico los residuos.
residuals_SARIMAX.plot(title="Residuals", ax=axes[0, 0])

# Grafico la densidad de los residuos.
residuals_SARIMAX.plot(kind='kde', title='Density', ax=axes[0, 1])

# Grafico la (ACF) de los residuos.
plot_acf(residuals_SARIMAX, ax=axes[1, 0], title='ACF')

# Grafico la (PACF) de los residuo.
plot_pacf(residuals_SARIMAX, ax=axes[1, 1], title='PACF')

# Muestro la figura con los graficos de los residuos.
plt.show()

In [ ]:
# Realizo predicciones utilizando el modelo SARIMAX ajustado con la variable exogena.
prediction_SARIMAX = result_SARIMAX.forecast(len(df_test['Valor_IPC']), exog=df_test['Dolar_BNA'])

prediction_SARIMAX

In [ ]:
# Creo la figura del grafico.
fig = go.Figure()

# Agrego una linea de los datos originales al grafico.
fig.add_trace(go.Scatter(x=df_modelos.index, y=df_modelos['Valor_IPC'], mode='lines', name='Datos originales', line=dict(color='steelblue')))

# Agrego una linea de la prediccion al grafico.
fig.add_trace(go.Scatter(x=df_test.index, y=prediction_SARIMAX, mode='lines', name='Predicción', line=dict(color='green')))

# Actualizo el disenio del grafico.
fig.update_layout(title='Variación mensual del Nivel general del IPC - Total Nacional', xaxis_title='', yaxis_title='Variación mensual')

# Muestro el grafico interactivo.
fig.show()

# 4. Data Mapping

In [ ]:
# Filtro para ver el historico anio por anio. (Indice del IPC)
df_historico = df_indices[(df_indices['Fecha'] == '2017-05-01') | (df_indices['Fecha'] == '2018-05-01') | (df_indices['Fecha'] == '2019-05-01') | (df_indices['Fecha'] == '2020-05-01') | (df_indices['Fecha'] == '2021-05-01') | (df_indices['Fecha'] == '2022-05-01') | (df_indices['Fecha'] == '2023-05-01')].copy()

df_historico

In [ ]:
# Cargo los datos geograficos del INDEC de las regiones estadisticas de Argentina desde el archivo GeoJSON.
df_geo = gpd.read_file('../Data/ARG_Regiones_INDEC.geojson')

# Filtro los datos por la columna 'Division_IPC'.
df_historico = df_historico[df_historico['Division_IPC'] == 'Nivel general']

# Uno los datos geograficos y los datos de indice del IPC por region.
df_mapeo = df_geo.merge(df_historico, on='Region')

# Coordenadas del centro de Argentina.
latitud_centro = -34.6037
longitud_centro = -58.3816

# Creo el grafico con barra de tiempo.
fig = px.choropleth(df_mapeo,
                    geojson=df_mapeo.geometry,
                    locations=df_mapeo.index,
                    color='Valor_IPC',
                    color_continuous_scale="Reds",
                    animation_frame=df_mapeo['Fecha'].dt.strftime('%b %y'),
                    title='Variación interanual del Nivel general del IPC por región',
                    labels={'Valor_IPC': 'Variación interanual', 'animation_frame': 'Fecha'},
                    template='plotly_dark',
                    hover_data=['Region'])

# Configuro la barra de tiempo y de valores.
fig.update_layout(coloraxis_colorbar=dict(x=1.05, len=0.5, yanchor="middle", y=0.5))

# Configuro el centro y el zoom del mapa.
fig.update_geos(center=dict(lon=longitud_centro, lat=latitud_centro), projection_scale=6, fitbounds="locations")

# Configuro el tamanio del mapa.
fig.update_layout(height=800, width=1000)

# Muestro el grafico.
fig.show()

**Nota:** No correr esta parte del proyecto porque tarda mucho en ejecutarse y se traba todo el código de la notebook.